In [ ]:
pip install pandas_datareader

In [ ]:
pip install yfinance 

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import os

#get the current working directory
dirname = os.getcwd()
nasdaq_files = os.path.join(dirname, 'data/stock_ticker_datasets/nasdaq.csv')
nyse_files = os.path.join(dirname, 'data/stock_ticker_datasets/nyse.csv')
hkex_files = os.path.join(dirname, 'data/stock_ticker_datasets/hkex.csv')


nasdaq=pd.read_csv(nasdaq_files) 
nyse=pd.read_csv(nyse_files)
hkex=pd.read_csv(hkex_files) 

# extract the ticker sybmbol from nasdaq
nasdaq['Symbol']=nasdaq['Symbol'].astype(str)
nasdaq_input=nasdaq['Symbol']

# extract the ticker sybmbol from nyse
nyse['Symbol']=nyse['Symbol'].astype(str)
nyse_input=nyse['Symbol']

# extract the ticker sybmbol from hkex 
hkex['Ticker']=hkex['Ticker'].astype(str)
hkex_input=hkex['Ticker']

#chunk row size
n = 100  

#split the datasets into smaller chunk size to prevent overrloading the API too frequently
nasdaq_df = [nasdaq_input[i:i+n] for i in range(0,nasdaq_input.shape[0],n)]
nyse_df = [nyse_input[i:i+n] for i in range(0,nyse_input.shape[0],n)]
hkex_df = [hkex_input[i:i+n] for i in range(0,hkex_input.shape[0],n)]


In [ ]:
from datetime import date
from pandas_datareader import data
import csv


# get the acutal price movement of US tickers
def get_todaylabel (start_date, end_date,ticker,perc_change):

    try: 
        info = data.get_data_yahoo(ticker,start_date,end_date)
        info['label']=(info['Close']-info['Open'])/info['Open']
        print(info['label'])
        path = os.path.join(dirname, 'data/stock_label/us_stock_label/data-'+ticker+'-label.csv')
        with open(path,'a') as f:
            for x in range(len(info)):
                newdate = str(info.index[x])
                newdate= newdate[0:10]

                if(info['label'][x]>perc_change):
                    label=2
                elif(info['label'][x]<-perc_change):
                    label=0
                elif( info['label'][x]>=-perc_change and info['label'][x]<=perc_change):
                    label=1
                print(label)
                writer = csv.writer(f)
                writer.writerow([newdate,label])
    except:
        print("error")
        pass
    
# get the acutal price movement of hk tickers
def get_hk_label (start_date,end_date,ticker):

    if(len(ticker)<4):
        in_ticker=ticker.zfill(4)
    elif(len(ticker)>=4):
        in_ticker=ticker
    path=in_ticker+'.HK'
    try:
        info = data.DataReader(path, 'yahoo', start_date,end_date)
        print(path)
        info['average']=(info['Close']-info['Open'])/info['Open']
        path = os.path.join(dirname, 'data/stock_label/hk_stock_label/data-'+ticker+'-label.csv')
        with open(path,'w') as f:
            for x in range(len(info)):
                newdate = str(info.index[x])
                newdate= newdate[0:10]
                average=info['average'][x]

                print(average)
                writer = csv.writer(f)
                writer.writerow([newdate,average])
         
    except:
        print("error")
        pass

In [ ]:
# collect all the price information of tickers in hkex
for hk in hkex_df:
    for name in hk:
        get_hk_label('2020-01-14','2021-02-07',name)

In [ ]:
# collect all the price information of tickers in nasdaq
for list in nasdaq_df:
    for name in list:
        get_todaylabel('2021-01-09','2021-01-11',name)

# collect all the price information of tickers in nyse
for list in nyse_df:
    for name in list:
        get_todaylabel('2021-01-09','2021-01-11',name)


In [ ]:
import pandas as pd
label= pd.DataFrame()

# join all the ticker's price change in nasdaq in a single csv
for df_list in nasdaq_df:
    for name in df_list:
        try:
            path=os.path.join(dirname,'data/stock_label/us_stock_label/'+'data-'+name+'-label.csv')
            df_dates=pd.read_csv(path,names=['dates','label'])
            ticker=[name for i in range(len(df_dates))]
            df_dates['ticker']=ticker
            label=label.append(df_dates)
            print(label)
        except:
            print('no file')
            pass



path= os.path.join(dirname, 'data/train-data/nasdaq/nasdaq_labelled_data.csv')
label.to_csv(path)

In [ ]:
label= pd.DataFrame()

# join all the ticker's price change in nyse in a single csv
for df_list in nyse_df:
    for name in df_list:
        try:
            path=os.path.join(dirname,'data/stock_label/us_stock_label/'+'data-'+name+'-label.csv')
            df_dates=pd.read_csv(path,names=['dates','label'])
            ticker=[name for i in range(len(df_dates))]
            df_dates['ticker']=ticker
            label=label.append(df_dates)
            print(label)
        except:
            print('no file')
            pass
        
path= os.path.join(dirname, 'data/train-data/nyse/nyse_labelled_data.csv')
label.to_csv(path)

In [ ]:
# join all the ticker's price change in hkex in a single csv
label= pd.DataFrame()
for df_list in hkex_df:
    for name in df_list:
        try:
            zname=name.zfill(4)
            path=os.path.join(dirname,'data/stock_label/hk_stock_label/'+'data-'+name+'-label.csv')
            df_dates=pd.read_csv(path,names=['dates','ticker_average'])
            ticker=[name for i in range(len(df_dates))]
            df_dates['ticker']=ticker
            label=label.append(df_dates)
            print(label)
        except:
            print('no file')
            pass
path= os.path.join(dirname, 'data/train-data/hkex/hkex_labelled_data.csv')
label.to_csv(path)

In [ ]:
# get the daily percentage change of hangseng index
import pandas as pd
import yfinance as yf
hkex_data = yf.Ticker('^HSI')
hkex_ticker = hkex_data.history(period='1d', start='2020-1-1', end='2021-2-7')
df = pd.DataFrame(hkex_ticker).reset_index()
merge= pd.DataFrame()
merge['dates']=df['Date']
merge['hsi_average']=(df['Close']-df['Open'])/df['Open']


path=os.path.join(dirname,'data/train-data/hkex/hsi_movement.csv')
merge.to_csv(path)
print(merge)

In [ ]:
import csv

# add the ticker_average and hsi_average column to the database
def merge_hk_label(no_label, label_data,merge_path):
    news_data=pd.read_csv(no_label)
    
    news_data.set_index(keys = ["dates"],inplace=True)

    label_data=pd.read_csv(label_data)
    label_data.set_index(keys = ["dates"],inplace=True)
    
    merge=pd.merge(news_data,label_data, how='inner', left_index=True, right_index=True)
    print(merge)
    final_label=[]

    for i in range(len(merge)):
        change= (merge['ticker_average'].iloc[i] - merge['hsi_average'].iloc[i])/ merge['hsi_average'].iloc[i]
        if change >0.01:
            final_label.append(2)
        elif change< 0.01:
            final_label.append(0)
        elif (merge['ticker_average'].iloc[i]>=-0.01 and merge['ticker_average'].iloc[i]<=0.01):
            final_label.append(1)
       
    merge['actual_label']=final_label
    print(merge)
    merge.to_csv(merge_path)

no_label= os.path.join(dirname, 'data/train-data/hkex/hkex_labelled_data.csv')
    
label_data=os.path.join(dirname,'data/train-data/hkex/hsi_movement.csv')

merge_path=os.path.join(dirname, 'data/train-data/hkex/hkex_merge_data.csv')

merge_hk_label(no_label,label_data,merge_path)
